In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from treeinterpreter import treeinterpreter as ti

In [7]:
with open('./pickle/Brownspot_lap0.pickle', 'rb') as handle:
    Brownspot_lap0 = pickle.load(handle)
    
with open('./pickle/Brownspot_lap1.pickle', 'rb') as handle:
    Brownspot_lap1 = pickle.load(handle)

with open('./pickle/Brownspot_lap2.pickle', 'rb') as handle:
    Brownspot_lap2 = pickle.load(handle)
    
with open('./pickle/Healthy_lap0.pickle', 'rb') as handle:
    Healthy_lap0 = pickle.load(handle)
    
with open('./pickle/Healthy_lap1.pickle', 'rb') as handle:
    Healthy_lap1 = pickle.load(handle)

with open('./pickle/Healthy_lap2.pickle', 'rb') as handle:
    Healthy_lap2 = pickle.load(handle)
    
with open('./pickle/sum_of_contoursArea_BrownSpot.pickle', 'rb') as handle:
    sum_of_contoursArea_BrownSpot = pickle.load(handle)

with open('./pickle/sum_of_contoursArea_Healthy.pickle', 'rb') as handle:
    sum_of_contoursArea_Healthy = pickle.load(handle)
    
with open('./pickle/secondLargestArea_BrownSpot.pickle', 'rb') as handle:
    secondLargestArea_BrownSpot = pickle.load(handle)

with open('./pickle/secondLargestArea_Healthy.pickle', 'rb') as handle:
    secondLargestArea_Healthy = pickle.load(handle)

In [8]:
data = []

data_level = []

for i in range(0, len(Brownspot_lap0)):
    data.append([Brownspot_lap0[i], Brownspot_lap1[i], Brownspot_lap2[i], sum_of_contoursArea_BrownSpot[i], secondLargestArea_BrownSpot[i]])
    data_level.append(0)
    data.append([Healthy_lap0[i], Healthy_lap1[i], Healthy_lap2[i], sum_of_contoursArea_Healthy[i], secondLargestArea_Healthy[i]])
    data_level.append(1)
    
print(len(data_level))
print(len(data))

1046
1046


In [9]:
ndata = np.array(data)
ndata_level = np.array(data_level)

In [10]:
df=pd.DataFrame({
    'laplacian red':ndata[:,0],
    'laplacian green':ndata[:,1],
    'laplacian blue':ndata[:,2],
    'sum of contours area':ndata[:,3],
    'second largest area':ndata[:, 4],
    'class':ndata_level[:]
})

In [11]:
X=df[['laplacian red', 'laplacian green', 'laplacian blue', 'sum of contours area', 'second largest area']]  # Features
y=df['class']  # Labels

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = joblib.load("random_forest_model.pkl")

# Global interpreter

In [14]:
pd.DataFrame({'Variable':X_train.columns,'Importance':model.feature_importances_}).sort_values('Importance', ascending=False)

,Variable,Importance
2,laplacian blue,0.380751
1,laplacian green,0.258484
4,second largest area,0.190786
3,sum of contours area,0.094294
0,laplacian red,0.075685


# Local interpreter

In [15]:
X_test[:10]

,laplacian red,laplacian green,laplacian blue,sum of contours area,second largest area
773,9.002501e+08,7.647619e+08,4.863958e+08,3958.0,120.5
280,6.236210e+08,1.021138e+09,2.442687e+08,9053.5,3756.0
629,3.378426e+08,1.630874e+08,5.155047e+08,616.5,9.0
318,1.641944e+09,1.721734e+09,4.675895e+08,22684.5,8058.5
1008,1.598758e+08,6.181821e+07,4.505696e+07,452.0,77.5
912,2.812600e+08,1.997553e+08,4.125189e+08,13489.0,6621.0
423,7.270573e+08,5.122326e+08,1.482513e+09,2958.0,219.5
231,1.208226e+09,5.146671e+08,2.008415e+09,2488.0,236.5
458,5.142225e+08,8.507511e+08,2.097508e+08,7716.0,1652.0
549,4.061721e+08,2.568544e+08,6.555993e+08,215.0,16.5


In [16]:
y_test[:10]

773     1
280     0
629     1
318     0
1008    0
912     0
423     1
231     1
458     0
549     1
Name: class, dtype: int32

In [17]:
instanceBrownspot = X_test[1:2]
instanceHealthy = X_test[2:3]

In [18]:
instanceBrownspot

,laplacian red,laplacian green,laplacian blue,sum of contours area,second largest area
280,623621034.0,1.021138e+09,244268729.0,9053.5,3756.0


In [19]:
instanceHealthy

,laplacian red,laplacian green,laplacian blue,sum of contours area,second largest area
629,337842603.0,163087382.0,515504703.0,616.5,9.0


In [20]:
print("----Local Interpretation for predicting BrownSpot Image")
prediction, bias, contributions = ti.predict(model, instanceBrownspot)
print (prediction)
print (bias)
#print "Feature contributions:"
for c, feature in zip(contributions[0], X_test.columns):
    print (feature, c)

----Local Interpretation for predicting BrownSpot Image
[[0.96686103 0.03313897]]
[[0.48839713 0.51160287]]
laplacian red [ 0.02405098 -0.02405098]
laplacian green [ 0.19056941 -0.19056941]
laplacian blue [ 0.13321194 -0.13321194]
sum of contours area [ 0.02831206 -0.02831206]
second largest area [ 0.10231951 -0.10231951]


In [45]:
print("----Local Interpretation for predicting Healthy Image")
prediction, bias, contributions = ti.predict(model, instanceHealthy)
print (prediction)
print (bias)
#print "Feature contributions:"
for c, feature in zip(contributions[0], X_test.columns):
    print (feature, c)

----Local Interpretation for predicting Healthy Image
[[0.14460057 0.85539943]]
[[0.48839713 0.51160287]]
laplacian red [-0.05312897  0.05312897]
laplacian green [-0.12980002  0.12980002]
laplacian blue [ 0.03037329 -0.03037329]
sum of contours area [-0.04717685  0.04717685]
second largest area [-0.14406401  0.14406401]


In [42]:
prediction.shape

(1, 2)